In [26]:
import simpy
import numpy as np
import functools
import matplotlib.pyplot as plt
from Stats import Entity, Resource, Source

## Description of problem: 

You have to model a restaurant. Arrivals come in poisson distribution

They must order. The Kitchen is described as follows: 
 - 2 cooks
 - 3 things to order:
   - fish -- 15 mins to prepare
   - burger -- 5 mins to prepare
   - salad -- 8 mins to prepare
   
Then must wait for a seat
- people will sit for between 20-30 minutes. 
- Capacity: 10
- Assume all attendees are dining alone (no need to worry about specific party capacity).


In [27]:
# First, set up interarrival generators and classes to represent resources
# for all of your problem parameters

# To use simpy is to understand generators
class Kitchen(Resource):
    def __init__(self, env, *args, **kwargs):
        self.name = "kitchen"
        self.env = env
        super().__init__(env, *args, **kwargs)
    
    def process_entity(self, entity):
        """
        logic for dealing with prep times for different foods
        """
        request = entity.request_resource(self)
        print(f'{entity.name} requesting Chef: {self.env.now}')
        yield request
        
        print(f'{entity.name} started Cooking: {self.env.now}')        
        entity.start_service_at_resource(self)
        yield self.env.timeout(self.next_service_time(entity))
    
    def next_service_time(self, entity):
        """
        Defines preparation time for each dish
        """
        if entity.attributes["food"] == "Fish":
            return 15
        elif entity.attributes["food"] == "Burger":
            return 5
        elif entity.attributes["food"] == "Salad":
            return 8
        else:
            raise Exception("Entity did not have food attribute defined")
        
 
class DiningRoom(Resource):
    def __init__(self, env, *args, **kwargs):
        self.name = "DiningRoom"
        self.env = env
        super().__init__(env, *args, **kwargs)
    
    def process_entity(self, entity):
        """
        logic for requesting a resource and 
        """
        request = entity.request_resource(self)
        print(f'{entity.name} requesting Table: {self.env.now}')
        yield request
        
        print(f'{entity.name} started Seating At Table: {self.env.now}')        
        entity.start_service_at_resource(self)
        yield self.env.timeout(self.next_service_time(entity))
    
    def next_service_time(self, entity):
        """
        logic for calculating next service time for resource
        """
        return np.radom.randint(20, 30) # 20 to 30 minute seating per entity

class CustomerSource(Source):
    
    

In [25]:
np.random.randint(20, 30)

24

In [ ]:
def dispose(entity):
    return lambda _: entity.dispose() 

def process(env, source, resource):
    """ Generate arrivals into our simulation
    
    Arguments:   
     env:  Our simulation environment
     source: the source of entities for our simulation
     resource: Shared resource
    """
    
    for arrival_time, entity in source.next_entity():
        yield arrival_time
        processed_entity = env.process(resource.process_entity(entity))
        processed_entity.callbacks.append(dispose(entity))
                